In [1]:
import torch
import numpy as np
import random
import pandas as pd
import os
import torch
import torch.nn.functional as F

import torch.nn as nn
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt
from torchvision import datasets,models, transforms
from torch.utils.data import DataLoader

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [5]:
import torch
import numpy as np
import random

def set_seed(seed):
    # Set seed for Python random module
    random.seed(seed)
    # Set seed for NumPy
    np.random.seed(seed)
    # Set seed for PyTorch
    torch.manual_seed(seed)
    # If using GPU, set the seed for CUDA
    torch.cuda.manual_seed(seed)
    # Ensures deterministic behavior in convolutional layers
    torch.backends.cudnn.deterministic = True
    # Disable certain optimizations for reproducibility
    torch.backends.cudnn.benchmark = False

# Set seed
set_seed(18)

In [7]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=20),
    transforms.RandomResizedCrop(size=224, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
valid_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the  datasets
train_dataset = datasets.ImageFolder(root="C:/Users/shrav/OneDrive/Documents/nndl/Dog_X_ray/Dog_heart/Train", transform=train_transform)
valid_dataset = datasets.ImageFolder(root="C:/Users/shrav/OneDrive/Documents/nndl/Dog_X_ray/Dog_heart/Valid", transform=valid_transform)

# Create a DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=True)

# Check dataset length and class names
print(f"Number of samples in training set: {len(train_dataset)}")
print(f"Class names: {train_dataset.classes}")
print(f"Number of samples in validation set: {len(valid_dataset)}")

Number of samples in training set: 1400
Class names: ['Large', 'Normal', 'Small']
Number of samples in validation set: 200


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


# Define a custom CNN model
class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, num_classes)
        
        # Dropout for regularization
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        
        # Flatten the tensor
        x = x.view(-1, 128 * 28 * 28)  # Flatten the feature map to a vector
        
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Instantiate the model
num_classes = len(train_dataset.classes)
model = CustomCNN(num_classes)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)  # Weight decay for L2 regularization

# Train the model
for epoch in range(100):  # Adjust epochs as needed
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate accuracy
        _, preds = torch.max(outputs, 1)
        correct_preds += torch.sum(preds == labels).item()
        total_preds += labels.size(0)

    train_acc = correct_preds / total_preds * 100
    print(f'Epoch [{epoch+1}/30], Loss: {running_loss/len(train_loader):.4f}, Train Accuracy: {train_acc:.2f}%')

    # Validation phase
    model.eval()
    correct_preds = 0
    total_preds = 0
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct_preds += torch.sum(preds == labels).item()
            total_preds += labels.size(0)

    valid_acc = correct_preds / total_preds * 100
    print(f'Validation Accuracy: {valid_acc:.2f}%')
    if train_acc>93:
        break



Epoch [1/30], Loss: 1.3092, Train Accuracy: 42.79%
Validation Accuracy: 44.50%
Epoch [2/30], Loss: 0.9778, Train Accuracy: 45.50%
Validation Accuracy: 43.00%
Epoch [3/30], Loss: 0.9589, Train Accuracy: 45.93%
Validation Accuracy: 45.00%
Epoch [4/30], Loss: 0.9417, Train Accuracy: 46.93%
Validation Accuracy: 44.50%
Epoch [5/30], Loss: 0.9461, Train Accuracy: 44.71%
Validation Accuracy: 49.00%
Epoch [6/30], Loss: 0.9348, Train Accuracy: 45.36%
Validation Accuracy: 47.50%
Epoch [7/30], Loss: 0.9140, Train Accuracy: 48.00%
Validation Accuracy: 55.50%
Epoch [8/30], Loss: 0.8885, Train Accuracy: 52.86%
Validation Accuracy: 58.00%
Epoch [9/30], Loss: 0.8597, Train Accuracy: 54.14%
Validation Accuracy: 52.00%
Epoch [10/30], Loss: 0.8538, Train Accuracy: 53.93%
Validation Accuracy: 57.50%
Epoch [11/30], Loss: 0.8400, Train Accuracy: 54.36%
Validation Accuracy: 58.00%
Epoch [12/30], Loss: 0.8142, Train Accuracy: 57.86%
Validation Accuracy: 59.00%
Epoch [13/30], Loss: 0.8060, Train Accuracy: 58.2

In [11]:
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss,
}, 'NNDL1.pth')

In [15]:
NNDL1 = torch.load('NNDL1.pth')
model.load_state_dict(NNDL1['model_state_dict'])
optimizer.load_state_dict(NNDL1['optimizer_state_dict'])
epoch = NNDL1['epoch']  # Start from the next epoch

C:\Users\shrav\AppData\Local\Temp\ipykernel_12032\3316447993.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  NNDL1 = torch.load('NNDL1.pth')


In [27]:
for epoch in range(201,230):  # Adjust epochs as needed
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate accuracy
        _, preds = torch.max(outputs, 1)
        correct_preds += torch.sum(preds == labels).item()
        total_preds += labels.size(0)

    train_acc = correct_preds / total_preds * 100
    print(f'Epoch [{epoch+1}], Loss: {running_loss/len(train_loader):.4f}, Train Accuracy: {train_acc:.2f}%')

    # Validation phase
    model.eval()
    correct_preds = 0
    total_preds = 0
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct_preds += torch.sum(preds == labels).item()
            total_preds += labels.size(0)

    valid_acc = correct_preds / total_preds * 100
    print(f'Validation Accuracy: {valid_acc:.2f}%')
    if train_acc>93:
        break

Epoch [202], Loss: 0.3594, Train Accuracy: 84.71%
Validation Accuracy: 68.50%
Epoch [203], Loss: 0.3652, Train Accuracy: 84.86%
Validation Accuracy: 64.00%
Epoch [204], Loss: 0.3383, Train Accuracy: 85.64%
Validation Accuracy: 69.50%
Epoch [205], Loss: 0.3459, Train Accuracy: 84.79%
Validation Accuracy: 67.50%
Epoch [206], Loss: 0.3486, Train Accuracy: 84.71%
Validation Accuracy: 68.00%
Epoch [207], Loss: 0.3449, Train Accuracy: 84.36%
Validation Accuracy: 68.50%
Epoch [208], Loss: 0.3449, Train Accuracy: 85.14%
Validation Accuracy: 69.50%
Epoch [209], Loss: 0.3384, Train Accuracy: 84.86%
Validation Accuracy: 71.00%
Epoch [210], Loss: 0.3249, Train Accuracy: 86.57%
Validation Accuracy: 69.00%
Epoch [211], Loss: 0.3588, Train Accuracy: 84.00%
Validation Accuracy: 69.50%
Epoch [212], Loss: 0.3126, Train Accuracy: 86.64%
Validation Accuracy: 68.00%
Epoch [213], Loss: 0.3155, Train Accuracy: 86.57%
Validation Accuracy: 68.00%
Epoch [214], Loss: 0.3443, Train Accuracy: 86.29%
Validation Acc

In [25]:
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss,
}, 'NNDL2.pth')

In [33]:

from PIL import Image
from torchvision import transforms
import os

# Directory containing test images (without subfolders)
test_dir = "C:/Users/shrav/OneDrive/Documents/nndl/Dog_X_ray/Dog_heart/Test"

# Transformation for test images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load images
def load_images_from_directory(directory):
    images = []
    file_names = []
    for file in os.listdir(directory):
        if file.endswith(('.png', '.jpg', '.jpeg')):  # Valid image formats
            file_path = os.path.join(directory, file)
            image = Image.open(file_path).convert('RGB')
            images.append(transform(image))  # Apply transformation
            file_names.append(file)  # Save the file name
    return images, file_names

images, file_names = load_images_from_directory(test_dir)

In [40]:
images_tensor = torch.stack(images).to(device)

# Get predictions
model.eval()
with torch.no_grad():
    outputs = model(images_tensor)
    _, predictions = torch.max(outputs, 1)

In [49]:
predictions

tensor([0, 0, 2, 2, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 2, 1, 2, 1, 0, 1, 1, 2, 0,
        0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 2, 0, 2, 1, 1, 2, 0, 0, 0, 1, 0, 1,
        1, 1, 1, 1, 1, 0, 1, 1, 0, 2, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1,
        0, 0, 1, 0, 2, 1, 1, 1, 1, 1, 2, 0, 0, 2, 1, 2, 0, 2, 1, 0, 1, 1, 1, 2,
        1, 1, 0, 0, 1, 1, 0, 0, 2, 0, 0, 0, 2, 2, 2, 0, 0, 1, 1, 1, 0, 0, 1, 1,
        0, 0, 1, 2, 1, 1, 2, 1, 1, 0, 0, 1, 2, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
        0, 1, 0, 2, 2, 2, 2, 1, 0, 0, 1, 1, 1, 0, 0, 1, 2, 0, 1, 0, 1, 1, 0, 2,
        1, 1, 1, 1, 1, 0, 2, 0, 1, 0, 1, 1, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 1, 1, 2, 2, 0, 2, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
        0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 2, 1, 2, 0, 1, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 2, 1, 1, 0, 0, 1, 1, 1, 0, 1, 2, 0,
        1, 2, 2, 1, 0, 1, 2, 1, 2, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 2, 1, 1, 0, 1, 0, 1, 0, 0, 1,

In [55]:
cpu_pred=predictions.cpu().numpy()
final=pd.DataFrame()
final['filename']=file_names
final['label']=cpu_pred

In [59]:
final.head()

,filename,label
0,100.png,0
1,1621.png,0
2,1622.png,2
3,1623.png,2
4,1624.png,2


In [61]:
final.to_csv('predctionscnn1.csv',index=False,header=False)

In [63]:
torch.save(model, "model_cnn1.pt")


In [67]:
# Load the model directly
model = torch.load("model_cnn1.pt")
model.eval()



C:\Users\shrav\AppData\Local\Temp\ipykernel_12032\881385825.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("model_cnn1.pt")


CustomCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=100352, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=3, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)